In [10]:
# import image utilities
import urllib.request
from PIL import Image
import numpy as np
import cv2

In [11]:
def resize_image(image):
    (h, w) = image.shape[:2]

    if w > h:
        ratio = 299.0 / w
        x = 299
        y = int(h * ratio)
        dim = (x, y)
    else:
        ratio = 299.0 / h
        y = 299
        x = int(w * ratio)
        dim = (x, y)

    # perform the actual resizing of the image
    resized = cv2.resize(image, dim, interpolation=cv2.INTER_LINEAR)

    if resized.shape[0] > resized.shape[1]:
        # numpy로 검은색 배경을 생성
        background = np.zeros((resized.shape[0], resized.shape[0], 3), dtype=np.uint8)
        # 이미지를 가운데로 겹치도록 설정
        x_offset = y_offset = resized.shape[0] // 2 - (resized.shape[1] // 2)
        # 해당 위치에 이미지를 겹침
        background[:, x_offset : x_offset + resized.shape[1]] = resized
    else:
        # 가로가 더 길 경우
        background = np.zeros((resized.shape[1], resized.shape[1], 3), dtype=np.uint8)
        x_offset = y_offset = resized.shape[1] // 2 - (resized.shape[0] // 2)
        background[y_offset : y_offset + resized.shape[0], :] = resized
    return background

In [12]:
f = open("/home/azure/ai/s3_file.txt", "r")
names = f.readlines()

for name in names:
    url = f"https://codiba.s3.us-east-2.amazonaws.com/{name}"
    img_name, img_format = name.split("/")[1].split(".")
    img_format = img_format.replace("\n", "")
    if img_format == "png":
        img = Image.open(urllib.request.urlopen(url)).convert("RGB")
    else:
        img = Image.open(urllib.request.urlopen(url))

    img = np.array(img)
    img = resize_image(img)
    img.save("/home/azure/ai/thumbnail", f"{name}")

AttributeError: 'numpy.ndarray' object has no attribute 'save'